![lop](../../../images/logo_diive1_128px.png)

<span style='font-size:40px; display:block;'>
<b>
    FCH4 vs. USTAR
</b>
</span>

---
**Notebook version**: `1` (19 Nov 2023)  
**Author**: Lukas Hörtnagl (holukas@ethz.ch)

# Background
- This notebook investigates the eddy covariance CH4 flux (nmol m-2 s-1) in relation to USTAR (m s-1).
- The used dataset is a special version of the [FLUXNET-CH4 Community Product](https://fluxnet.org/data/fluxnet-ch4-community-product/).
- `FCH4` in this notebook is the flux **before** USTAR filtering.

# Setup

In [ ]:
# Variable names in data files
USTAR1 = 'USTAR'
USTAR2 = 'USTAR_1_1_1'  # Alternative name in some files
FLUX1 = 'FCH4'
FLUX2 = 'FCH4_1_1_1'  # Alternative name in some files

# Source folder with data files in parquet format
SOURCEDIR = r"data/CH4/FLUXNET-CH4 Community Product Special Version preUSTAR/PARQUET"

# Imports

In [ ]:
import importlib.metadata
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
import numpy as np
import pandas as pd

from diive.core.io.filereader import search_files
from diive.core.io.files import load_parquet
from diive.core.plotting.scatter import ScatterXY

version_diive = importlib.metadata.version("diive")
print(f"diive version: v{version_diive}")

# Search data files
- All data files were first converted to the [Parquet](https://parquet.apache.org/) format for faster processing.

In [ ]:
foundfiles = search_files(searchdirs=[SOURCEDIR], pattern='*.parquet')
# foundfiles = foundfiles[0:11]
[print(f"{ix}: {f}") for ix, f in enumerate(foundfiles)];

# Determine required number of rows for large figure

In [ ]:
n_files = len(foundfiles)
plots_per_row = 4

# Plotting function (with bins)

In [ ]:
def plot(foundfiles: list, patchid: int):
    fig = plt.figure(facecolor='white', figsize=(28.8, 21.6), dpi=72)
    gs = gridspec.GridSpec(3, 4)  # rows, cols   
    gs.update(wspace=0.15, hspace=0.25, left=0.03, right=0.97, top=0.97, bottom=0.03)
    
    current_row = 0
    col = 0
    axes = {} 
    for ix, ff in enumerate(foundfiles):
        site = ff.name.replace('AMF_', '').split('_')[0]
        df = load_parquet(ff)
       
        if col > plots_per_row-1:    
            col = 0
            current_row += 1        
        ax = fig.add_subplot(gs[current_row, col])        
        xcol = USTAR1 if USTAR1 in df.columns else USTAR2
        ycol = FLUX1 if FLUX1 in df.columns else FLUX2    
        ScatterXY(x=df[xcol], y=df[ycol], nbins=20, title=site, binagg='median', ax=ax, ylim='auto').plot()        
        col += 1   
    fig.savefig(f"FCH4vsUSTAR_{patchid}.png")

# Create plots (with bins)

In [ ]:
patch = 0
for firstfile in range(0, n_files, 12):    
    patch += 1
    lastfile = firstfile + 12
    print(f"Patch {patch} ({firstfile}-{lastfile})")
    plot(foundfiles=foundfiles[firstfile:lastfile], patchid=patch)    